In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 23.0 MB/s eta 0:00:00
  

In [13]:
from pickle import NONE
import pandas as pd

# 0 : Hate
# 1 : Offensive
# 2 : Neither

df = pd.read_csv('/content/twitter.csv')

df['label'] = df['class'].apply(lambda x: 1 if x==2 else 0)

df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,label
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,1
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0


In [6]:
from sklearn.model_selection import train_test_split

X=df['tweet'].values
Y=df['label'].values
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

train_data=[]
for i in range(len(X_train)):
  l=[X_train[i],Y_train[i]]
  train_data.append(l)

train_df = pd.DataFrame(train_data)
train_df.columns = ["tweet", "label"]



In [7]:
train_data=[]
for i in range(len(X_train)):
  l=[X_train[i],Y_train[i]]
  train_data.append(l)

train_df = pd.DataFrame(train_data)
train_df.columns = ["tweet", "label"]
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import f1_score
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = ClassificationArgs()
model_args.num_train_epochs=2
model_args.overwrite_output_dir=True
model_args.adafactor_eps=(1e-4,1e-3)
model_args.learning_rate=7e-5

model = ClassificationModel("bert", "bert-base-cased", args=model_args,use_cuda=True)
model.train_model(train_df,acc=f1_score)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/19826 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/2479 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/2479 [00:00<?, ?it/s]

(4958, 0.2240619157268521)

In [8]:
eval_data=[]
for i in range(len(X_test)):
  l=[X_test[i],Y_test[i]]
  eval_data.append(l)
print(len(eval_data))
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["tweet", "labels"]

result, model_outputs, wrong_predictions = model.eval_model(eval_df,acc=f1_score)
print(result)

4957


/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/4957 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/620 [00:00<?, ?it/s]

{'mcc': 0.8612675337830903, 'tp': 753, 'tn': 4008, 'fp': 116, 'fn': 80, 'auroc': 0.9646408514909358, 'auprc': 0.9047724622790587, 'acc': 0.8848413631022326, 'eval_loss': 0.16582901247086063}


In [9]:
# pred,_ = model.predict(["Hey,my phone is facing technical"])

pred = model.predict(["Good afternoon! How can I fuck you?","Hi, I'd like to inquire about your nigga plans.","Of course! What specific information are you looking for?","I want to know if you offer any discounts for annual subscriptions."])

print(pred)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

(array([0, 0, 1, 1]), array([[ 3.27148438, -2.65234375],
       [ 3.27148438, -2.65234375],
       [-2.13671875,  1.81640625],
       [-2.59570312,  1.96582031]]))


In [10]:
model.model.save_pretrained('modelprof')
model.tokenizer.save_pretrained('modelprof')
model.config.save_pretrained('modelprof/')

In [11]:
model = ClassificationModel("bert","modelprof")

pred = model.predict(["Good afternoon! How can I help you?","Hi, I'd like to inquire about your pricing plans.","Of course! What specific information are you looking for?","I want to know if you offer any discounts for annual subscriptions."])
print(pred)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

(array([1, 1, 1, 1]), array([[-2.078125  ,  1.75976562],
       [-2.359375  ,  1.90820312],
       [-2.13671875,  1.81640625],
       [-2.59570312,  1.96582031]]))
